__применение парсера объявлений циан.ru__

_Evgeny S. Borisov <parser@mechanoid.su>_

In [1]:
import sys
import re
import logging
from datetime import datetime as dtm
from tqdm.notebook import tqdm
import pandas as pd
from bs4 import BeautifulSoup
from time import sleep
from random import random

In [2]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.float_format', '{:.2f}'.format)
tqdm.pandas()

logging.basicConfig(
        format=u'[%(levelname)-8s] %(asctime)s | %(message)s',
        datefmt='%Y-%m-%d %H:%M:%S',
        level=logging.INFO,
        # level=logging.DEBUG,
        stream=sys.stdout,
    )

In [3]:
raw_data_path = 'data/raw/flat/cian' # каталог с файлами объявлений

In [4]:
profile_path = '/home/mechanoid/.mozilla/firefox/p144xo2m.default-release'
# base_url, req_param = 'https://sevastopol.cian.ru/kupit-kvartiru/', ''
base_url, req_param ='https://www.cian.ru/cat.php', 'deal_type=sale&offer_type=flat&region=184723'

In [5]:
%%time

# from lib.downloader import DownloaderSimple
# from lib.parser import AdsListParser
from lib.downloader import DownloaderSeleniumFirefox
from lib.cian import CianParser


df,html = CianParser(
        base_url = base_url,
        driver=DownloaderSeleniumFirefox(profile_path),
        # driver=DownloaderSimple(),
    ).load(req_param=req_param, keep_html=True) # , npage_start=53) # , page_limit=3 ) #

print(len(df))

[INFO    ] 2022-09-15 10:20:54 | DownloaderSeleniumFirefox: downloader init
[INFO    ] 2022-09-15 10:20:54 | DownloaderSeleniumFirefox: open virtual browser
[INFO    ] 2022-09-15 10:20:56 | AdsListParser: downloader init
[INFO    ] 2022-09-15 10:20:56 | CianParser: downloader init
[INFO    ] 2022-09-15 10:20:56 | AdsListParser: start read and parse pages...
[INFO    ] 2022-09-15 10:21:07 | AdsListParser: read page 1
[INFO    ] 2022-09-15 10:21:18 | AdsListParser: read page 2
[INFO    ] 2022-09-15 10:21:29 | AdsListParser: read page 3
[INFO    ] 2022-09-15 10:21:39 | AdsListParser: read page 4
[INFO    ] 2022-09-15 10:21:54 | AdsListParser: read page 5
[INFO    ] 2022-09-15 10:22:07 | AdsListParser: read page 6
[INFO    ] 2022-09-15 10:22:14 | AdsListParser: read page 7
[INFO    ] 2022-09-15 10:22:19 | AdsListParser: read page 8
[INFO    ] 2022-09-15 10:22:33 | AdsListParser: read page 9
[INFO    ] 2022-09-15 10:22:48 | AdsListParser: read page 10
[INFO    ] 2022-09-15 10:22:54 | AdsLis

In [6]:
df.sample(3)

,OfferTitle,OfferSubtitle,Deadline,MainPrice,PriceInfo,GeoLabel,TimeLabel,LinkArea,Description,page,ts
831,"2-комн. кв., 45,3 м², 4/5 этаж",,,6 300 000 ₽,139 073 ₽/м²,"[Севастополь, р-н Ленинский, мкр. Острякова, улица Хрусталева, 31]","[вчера, 13 сен, 22:56]",https://sevastopol.cian.ru/sale/flat/276418357/,"Продаю 2-комнатую квартиру, расположенная в Ленинском районе. Квартира требует ремонта. Деревянный оконные рамы, пол оргалит. Центральное отопление, газовая колонка, установлен кондиционер. Совмещенный санузел. Балкон застекленный. Окна выходят во двор.\n \nРасположение дома очень удобное. В пешей доступности огромный рынок 5-ого километра, где можно купить практически все! Рядом с домом гимназия, сквер 60-летия Октября и ТЦ Московский.\n\nВозможна покупка в ипотеку! Скорее звоните и записывайтесь на просмотр. . Номер в базе: 8084937.",30,2022-09-15 10:30:49.198554
44,"2-комн. кв., 68,8 м², 8/10 этаж",Сдача корпуса 3 кв. 2022,сдача ГК: 4 кв. 2022 года,10 935 760 ₽,158 950 ₽/м²,"[Севастополь, р-н Ленинский, спуск Стрелецкий]","[5 часов назад, сегодня, 04:26]",https://sevastopol.cian.ru/sale/flat/275272759/,"Статусный современный жилой комплекс бизнес-класса Центр находится в самом сердце города Федерального значения Севастополь. ЖК Центр жизнь в эпицентре событий. Расположение в историческом центре Ленинского района дарит возможность быстрого доступа до главных достопримечательностей города, важных социальных объектов, туристических и развлекательный мест, центральных набережных и пляжей.\nНа 1 этаже комплекса расположатся торгово-офисные помещения и ресторан. Три нижних этажа ЖК занимает подземный паркинг с видеонаблюдением, где Ваш автомобиль будет в целостности и сохранности.\nТри 10-этажных корпуса располагаются на едином стилобате это закрытый двор без машин под видеонаблюдением с детскими и спортивными площадками с безопасным покрытием, где Вы и Ваши дети могут чувствовать себя спокойно гуляя, играя и занимаясь спортом.\nХоллы комплекса выполнены по индивидуальному дизайн проекту, в каждом подъезде установлены бесшумные скоростные лифты, спускающиеся в паркинг.\nБольшинство квартир ЖК Центра видовые. В квартирах сделаны увеличенные оконные проемы либо установлены панорамные окна, что дарит много света и максимальный вид на панораму города, праздничные салюты, исторические достопримечательности и, конечно, морские пейзажи. \nКаждая квартира оборудована автономным двухконтурным газовым котлом, что экономит расходы на КУ и дает возможность контроля температуры в квартире под Ваши предпочтения.\nВ пешей доступности от комплекса лучшие гимназии города, несколько детских садов, поликлиник и больниц, спортивный оздоровительный комплекс 200 лет Севастополя, футбольное и баскетбольное поле, поле для Регби, , открытые теннисные корты, крытый бассейн, Ледовая арена для занятий фигурным катанием и хоккеем. В 5 минутах от комплекса находится рынок Чайка.",2,2022-09-15 10:30:49.198554
193,"2-комн. кв., 30 м², 2/2 этаж",,,10 800 000 ₽,360 000 ₽/м²,"[Севастополь, р-н Балаклавский, мкр. Балаклава, улица Калича, 49]","[вчера, вчера, 15:28]",https://sevastopol.cian.ru/sale/flat/264716471/,"Двухкомнатная квартира 30 м2 на 2/2 этаже дома. Расположена в г. Севастополь, Балаклавский район, ул. Калича, д. 49.\nПозвоните нам, чтобы узнать все детали! \nТехнические характеристики: \n \n-Адрес: г. Севастополь, Балаклавский район, ул. Калича, д. 49\n-Год постройки дома: 1917\n-Этаж /этажность: 2/2\n-Количество комнат: 2\n-Комнаты: смежные \n-Общая площадь: 30 м2\n-Жилая площадь: 21 м2\n-Площадь кухни: 3,5 м2\n-Высота потолка: 3,2 м \n-Санузел: совмещенный \n-Водоснабжение: центральное \n-Горячее водоснабжение: бойлер \n-Отопление: центральное \nДополнительная информация: \nПервая линия от нaбepежной. B шaгoвoй доcтупноcти городской пляж, дeтский сaд, мaгазины, oвощнoй pынoк, аптекa, ocтанoвки гopодcкoго трaнспopта.\nКваpтирa общей площадью 30 кв.м. наxодитcя в двухэтaжнoм дoме на втором этаже. Имеются две раздельные комнаты. Окна расположены на 

In [7]:
print(len(df))
df = pd.DataFrame(df).dropna().drop_duplicates(['LinkArea','Description',]).reset_index(drop=True)
print(len(df))

1509
1507


In [8]:
ts = dtm.now().strftime('%Y-%m-%d_%H-%M')
assert len(df)>0
# df.to_excel(f'{raw_data_path}/cian_{ts}_raw.xlsx',index=False)
df.to_pickle(f'{raw_data_path}/cian_{ts}_raw.pkl')

----

In [9]:
# df = pd.read_excel('data/raw/cian/cian_2022-09-08_13-49_raw.xlsx')

In [10]:
# df

In [11]:
# df['GeoLabel'].apply( lambda s : len(s) ) #', '.join(s) )

# s = ['Севастополь', 'р-н Гагаринский', 'Казачья Бухта мкр', 'Скифия ЖК']
# ', '.join(s)

In [12]:
# from lib.cian import CianDataCleaner

# CianDataCleaner().transform(df)

----

In [13]:
# with open('tmp/cian.html','wt') as f: f.write(html[0])